In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_d2e6127793d04cbdaf080d8269327416 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="https://iam.eu-de.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_d2e6127793d04cbdaf080d8269327416.get_object(Bucket='potraining-donotdelete-pr-zxxk56klnvpxz9',Key='_all_docs.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face an error during data loading.
# Please read the documentation of 'pandas.read_json()' and 'pandas.io.json.json_normalize' to learn more about the possibilities to adjust the data loading.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html

df_data_1 = pd.read_json(body, orient='values')
df_data_1.head()



,total_rows,offset,rows
0,418,0,"{'id': '01545c7e-a8ca-11e9-ac98-890c240138bb',..."
1,418,0,"{'id': '01ba337d-a8cb-11e9-9d24-890c240138bb',..."
2,418,0,"{'id': '021d9e1d-a8cc-11e9-8d6a-890c240138bb',..."
3,418,0,"{'id': '0282af91-a8cd-11e9-be37-890c240138bb',..."
4,418,0,"{'id': '02e4f741-a8ce-11e9-b476-890c240138bb',..."


In [2]:
from pandas.io.json import json_normalize 
data = json_normalize(df_data_1['rows']).filter(['doc.timestamp','doc.deviceId',
                                                 'doc.data.d.Power','doc.data.d.TurbineSpeed',
                                                 'doc.data.d.WindSpeed','doc.data.d.WindDirection',
                                                 'doc.data.d.VibrationX','doc.data.d.VibrationY'], axis=1)
data.head()


,doc.timestamp,doc.deviceId,doc.data.d.Power,doc.data.d.TurbineSpeed,doc.data.d.WindSpeed,doc.data.d.WindDirection,doc.data.d.VibrationX,doc.data.d.VibrationY
0,2019-07-17T21:35:17.067+02:00,Aerogenerador01,718.664666,47.829236,32.0,160.0,0.190032,0.191457
1,2019-07-17T21:42:27.231+02:00,Aerogenerador01,285.496363,19.131924,13.0,248.0,0.127026,0.126930
2,2019-07-17T21:49:37.380+02:00,Aerogenerador01,290.997999,19.268843,13.0,248.0,0.125184,0.124915
3,2019-07-17T21:56:47.539+02:00,Aerogenerador01,286.152119,19.103679,13.0,248.0,0.122965,0.125991
4,2019-07-17T22:03:57.680+02:00,Aerogenerador01,297.796633,19.748421,13.0,248.0,0.122616,0.125108


In [3]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,doc.data.d.Power,doc.data.d.TurbineSpeed,doc.data.d.WindSpeed,doc.data.d.WindDirection,doc.data.d.VibrationX,doc.data.d.VibrationY
doc.data.d.Power,1,0.999945,0.99921,-0.994028,0.993214,0.992605
doc.data.d.TurbineSpeed,0.999945,1,0.99926,-0.994059,0.993306,0.992737
doc.data.d.WindSpeed,0.99921,0.99926,1,-0.994966,0.994332,0.99362
doc.data.d.WindDirection,-0.994028,-0.994059,-0.994966,1,-0.995865,-0.995998
doc.data.d.VibrationX,0.993214,0.993306,0.994332,-0.995865,1,0.992803
doc.data.d.VibrationY,0.992605,0.992737,0.99362,-0.995998,0.992803,1
